In [3]:
import numpy as np
import pandas as pd

# model module
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# pipeline modules
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

# preprocessing modules
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from scipy import sparse
from itertools import combinations


class SparseInteractions(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2, feature_name_separator="_"):
        self.degree = degree
        self.feature_name_separator = feature_name_separator
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        if not sparse.isspmatrix_csc(X):
            X = sparse.csc_matrix(X)
            
        if hasattr(X, "columns"):
            self.orig_col_names = X.columns
        else:
            self.orig_col_names = np.array([str(i) for i in range(X.shape[1])])
            
        spi = self._create_sparse_interactions(X)
        return spi
    
    
    def get_feature_names(self):
        return self.feature_names
    
    def _create_sparse_interactions(self, X):
        out_mat = []
        self.feature_names = self.orig_col_names.tolist()
        
        for sub_degree in range(2, self.degree + 1):
            for col_ixs in combinations(range(X.shape[1]), sub_degree):
                # add name for new column
                name = self.feature_name_separator.join(self.orig_col_names[list(col_ixs)])
                self.feature_names.append(name)
                
                # get column multiplications value
                out = X[:, col_ixs[0]]    
                for j in col_ixs[1:]:
                    out = out.multiply(X[:, j])

                out_mat.append(out)

        return sparse.hstack([X] + out_mat)

** Summary **

Budgets for schools and school districts are huge, complex, and unwieldy. It's no easy task to digest where and how schools are using their resources. Education Resource Strategies is a non-profit that tackles just this task with the goal of letting districts be smarter, more strategic, and more effective in their spending.

Your task is a multi-class-multi-label classification problem with the goal of attaching canonical labels to the freeform text in budget line items. These labels let ERS understand how schools are spending money and tailor their strategy recommendations to improve outcomes for students, teachers, and administrators. 

In [4]:
df = pd.read_csv('TrainingData.csv', index_col=0)
df.head()

Function          Use          Sharing Reporting  \
134338     Teacher Compensation  Instruction  School Reported    School   
206341                 NO_LABEL     NO_LABEL         NO_LABEL  NO_LABEL   
326408     Teacher Compensation  Instruction  School Reported    School   
364634  Substitute Compensation  Instruction  School Reported    School   
47683   Substitute Compensation  Instruction  School Reported    School   

       Student_Type Position_Type               Object_Type     Pre_K  \
134338     NO_LABEL       Teacher                  NO_LABEL  NO_LABEL   
206341     NO_LABEL      NO_LABEL                  NO_LABEL  NO_LABEL   
326408  Unspecified       Teacher  Base Salary/Compensation  Non PreK   
364634  Unspecified    Substitute                  Benefits  NO_LABEL   
47683   Unspecified       Teacher   Substitute Compensation  NO_LABEL   

         Operating_Status            Object_Description  \
134338  PreK-12 Operating                           NaN   
206341      Non-Operating           CONTRACTOR SERVICES   
326408  PreK-12 Operating  Personal Services - Teachers   
364634  PreK-12 Operating             EMPLOYEE BENEFITS   
47683   PreK-12 Operating  TEACHER COVERAGE FOR TEACHER   

                    ...               Sub_Object_Description  \
134338              ...                                  NaN   
206341              ...                                  NaN   
326408              ...                                  NaN   
364634              ...                                  NaN   
47683               ...                                  NaN   

       Location_Description  FTE     Function_Description  \
134338                  NaN  1.0                      NaN   
206341                  NaN  NaN                 RGN  GOB   
326408                  NaN  1.0                      NaN   
364634                  NaN  NaN  UNALLOC BUDGETS/SCHOOLS   
47683                   NaN  NaN              NON-PROJECT   

       Facility_or_Department              Position_Extra      Total  \
134338                    NaN               KINDERGARTEN   50471.810   
206341                    NaN                UNDESIGNATED   3477.860   
326408                    NaN                     TEACHER  62237.130   
364634                    NaN  PROFESSIONAL-INSTRUCTIONAL     22.300   
47683                     NaN  PROFESSIONAL-INSTRUCTIONAL     54.166   

                   Program_Description        Fund_Description  \
134338                    KINDERGARTEN            General Fund   
206341   BUILDING IMPROVEMENT SERVICES                     NaN   
326408           Instruction - Regular  General Purpose School   
364634  GENERAL MIDDLE/JUNIOR HIGH SCH                     NaN   
47683    GENERAL HIGH SCHOOL EDUCATION                     NaN   

                               Text_1  
134338                            NaN  
206341  BUILDING IMPROVEMENT SERVICES  
326408                            NaN  
364634            REGULAR INSTRUCTION  
47683             REGULAR INSTRUCTION  

[5 rows x 25 columns]

** Preprocessing **

First, we get the data from the TrainingData.csv. Applying fuctions to string can be slow, so we utilize category type to speed up some pandas functions in those categorical datas. The problem description made things easier for us, telling us what columns is categorical.

In [5]:
# Labels that we can converto to categorical type
LABELS = ['Function', 'Use', 'Sharing', 'Reporting',
          'Student_Type', 'Position_Type', 'Object_Type',
          'Pre_K', 'Operating_Status']

df[LABELS] = df[LABELS].apply(lambda x: x.astype('category'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 25 columns):
Function                  400277 non-null category
Use                       400277 non-null category
Sharing                   400277 non-null category
Reporting                 400277 non-null category
Student_Type              400277 non-null category
Position_Type             400277 non-null category
Object_Type               400277 non-null category
Pre_K                     400277 non-null category
Operating_Status          400277 non-null category
Object_Description        375493 non-null object
Text_2                    88217 non-null object
SubFund_Description       306855 non-null object
Job_Title_Description     292743 non-null object
Text_3                    179964 non-null object
Text_4                    53746 non-null object
Sub_Object_Description    91603 non-null object
Location_Description      162054 non-null object
FTE                       126071 non

If we take a close look to the df.info() output, we can see that two columns are float type. Let's save those columns for later.

In [6]:
NUMERIC = ['FTE', 'Total']

# the label class we want to predict
label_dummies = pd.get_dummies(df[LABELS])

PATTERN = '[A-Za-z0-9]+(?=\\s+)'

In [7]:
# Our multilabel train test split
def multilabel_sample(y, size=1000, min_count=5, seed=None):
    
    #check if unique values of y is only 0 or 1 values
    try:
        if (np.unique(y).astype(int) != np.array([0,1])).all():
            raise ValueError()
    except (TypeError, ValueError):
        raise ValueError('Only binary indicator matrices allowed')
    
    if (y.sum(axis=0) < min_count).any():
        raise ValueError('Some classes do not have enough examples. Change min_count if necessary')
    
    if size <= 1:
        size = np.floor(y.shape[0] * size)
    
    if y.shape[1] * min_count > size:
        msg = " Size less than number os columns * min_count, returning {} items instead of {}"
        warn(msg.format(y.shape[1] * min_count, size))
        size = y.shape[1] * min_count
    
    rng = np.random.RandomState(seed if seed is not None else np.random.randint(1))
    
    if isinstance(y, pd.DataFrame):
        choices = y.index
        y = y.values
    else:
        choices = np.arange(y.shape[0])
    
    sample_idxs = np.array([], dtype=choices.dtype)
    
    for j in range(y.shape[1]):
        label_choices = choices[y[:, j] == 1]
        label_idxs_sampled = rng.choice(label_choices, size=min_count, replace=False)
        sample_idxs = np.concatenate([label_idxs_sampled, sample_idxs])

    sample_idxs = np.unique(sample_idxs)

    # now that we have at least min_count of each, we can just random sample
    sample_count = int(size - sample_idxs.shape[0])

    # get sample_count indices from remaining choices
    remaining_choices = np.setdiff1d(choices, sample_idxs)
    remaining_sampled = rng.choice(remaining_choices,
                                   size=sample_count,
                                   replace=False)

    return np.concatenate([sample_idxs, remaining_sampled])
        
        
def multilabel_train_test_split(X, Y, size, min_count=5, seed=None):
    
    index = Y.index if isinstance(Y, pd.DataFrame) else np.arange(Y.shape[0])
    
    test_set_idxs = multilabel_sample(Y, size=size, min_count=min_count, seed=seed)
    train_set_idxs = np.setdiff1d(index, test_set_idxs)

    test_set_mask = index.isin(test_set_idxs)
    train_set_mask = ~test_set_mask

    return (X[train_set_mask], X[test_set_mask], Y[train_set_mask], Y[test_set_mask])

# Evaluation Function
def compute_log_loss(pred, y_test, eps=1e-14):
    predict = np.clip(pred, eps, 1 - eps)
    loss = -1 * np.mean( y_test * np.log(predict) + \
              (1 - y_test) * np.log(1 - predict)) 
    return loss

def combine_text_columns(data_frame, to_drop=LABELS+NUMERIC):
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    text_data.fillna('', inplace=True)
    return text_data.apply(lambda x: " ".join(x), axis=1)

** Building a simple model **

As a general tip for this kind of competition, we'll build a  simple model and make a prediction to see how it goes. Since this is a MultiLabel problem (see http://scikit-learn.org/stable/modules/multiclass.html) we'll be using OneVSRestClassifier ensembled with LogisticRegression.

** Using text as feature **

We achieved **1.90** log loss score using only the two float columns as features. The other columns of this DataFrame is string data, thus, we cant use them as input for our model. We need to process these data.

In [8]:
# extract all the text data
text_data = combine_text_columns(df)

# class we want to predict
label_dummies = pd.get_dummies(df[LABELS])

#get the train and test dataset
X_train, X_test, y_train, y_test = multilabel_train_test_split(combine_text_columns(df),
                                                               label_dummies,
                                                               size=.2, 
                                                               seed=123)

# setting the pipeline
pl = Pipeline([ ('vect', CountVectorizer()),
                ('clf', OneVsRestClassifier(LogisticRegression())) 
              ])

pl.fit(X_train, y_train)
print("Log loss function result : {}".format(pl.score(X_test, y_test)))

Log loss score : 0.8533258384860408


** Using the whole dataset **

Until now, we only used a slice of the entire dataset. Now, we'll combine the text and the numeric features in order to build a more robust model. But, to use the pipeline function with the text AND the numeric data, we need to separate them in two, since pipeline do not know how to treat text and numeric data together.

In [8]:
get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC], validate=False)

process_and_join = FeatureUnion([
                    ('numeric_features', Pipeline([
                        ('selector', get_numeric_data),
                        ('imputer', Imputer())
                    ])),
                    ('text_features', Pipeline([
                        ('selector', get_text_data),
                        ('vec', CountVectorizer(stop_words='english'))
                    ]))]
                )

pl = Pipeline([
    ('union', process_and_join),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

NON_LABELS = [col for col in df.columns if col not in LABELS]

X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               label_dummies,
                                                               size=.2, 
                                                               seed=123)

#pl.fit(X_train, y_train)
#print('Log Loss function result: {}'.format(pl.score(X_test, y_test)))

** Testing a new model **

We've got a good result using both the text and the numeric data. Now, let's see if other model perform better than logistic regression.

In [32]:
from sklearn.ensembleble import RandomForestClassifier

get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC], validate=False)

process_and_join = FeatureUnion([
                    ('numeric_features', Pipeline([
                        ('selector', get_numeric_data),
                        ('imputer', Imputer())
                    ])),
                    ('text_features', Pipeline([
                        ('selector', get_text_data),
                        ('vec', CountVectorizer(token_pattern=PATTERN, 
                                                ngram_range=(1,2), 
                                                stop_words='english'))
                    ]))]
                )

pl = Pipeline([
    ('union', process_and_join),
    ('clf', RandomForestClassifier(n_estimators=15))
])

X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               label_dummies,
                                                               size=.2, 
                                                               seed=123)

pl.fit(X_train, y_train)
print('Log Loss function result: {}'.format(pl.score(X_test, y_test)))

Log Loss function result: 0.9134719880082444


Okay, nothing good here, let's stick to the logistic regression. But now, we'll get more serious :)

In [ ]:
get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC], validate=False)

chi_k = 300

process_and_join = FeatureUnion([
                    ('numeric_features', Pipeline([
                        ('selector', get_numeric_data),
                        ('imputer', Imputer())
                    ])),
                    ('text_features', Pipeline([
                        ('selector', get_text_data),
                        ('vec', HashingVectorizer(token_pattern=PATTERN,
                                                norm=None,
                                                non_negative=True,
                                                ngram_range=(1,2), 
                                                stop_words='english')),
                        ('dim_red', SelectKBest(chi2, chi_k))
                    ]))]
                )

pl = Pipeline([
    ('union', process_and_join),
    ('int', SparseInteractions(degree=2)),
    ('scale', MaxAbsScaler()),
    ('clf', OneVsRestClassifier(LogisticRegression()))
])

X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               label_dummies,
                                                               size=.2, 
                                                               seed=123)

pl.fit(X_train, y_train)
print('Log Loss function result: {}'.format(pl.score(X_test, y_test)))

In [ ]:
# laod the test dataset and create the submission.csv
holdout = pd.read_csv('TestData.csv', index_col=0)
prediction = pl.predict_proba(holdout[NUMERIC])
submission = pd.DataFrame(columns=label_dummies.columns, index=holdout.index, data=prediction)
submission.to_csv('submission.csv')
submission.head()